In [54]:
from opentrons import simulate
from opentrons.types import Point
from opentrons.drivers.rpi_drivers import gpio
from opentrons import protocol_api
import os
import json
import math
import time

ctx = simulate.get_protocol_api('2.1')

## Constants
NUM_SAMPLES = 50
REAGENT_LABWARE = 'nest 12 reservoir plate'
MAGPLATE_LABWARE = 'nest deep well plate'
WASTE_LABWARE = 'nest 1 reservoir plate'
ELUTION_LABWARE = 'opentrons aluminum biorad plate'
TIP_TRACK = True

## global vars
robot = None
tip_log = {}
tip_log['count'] = {}
tip_log['tips'] = {}
tip_log['max'] = {}

reagent_res = ctx.load_labware(
        'nest_12_reservoir_15ml', '7', 'reagent reservoir')
magdeck = ctx.load_module('magdeck', '10')
magplate = magdeck.load_labware('usascientific_96_wellplate_2.4ml_deep')

/home/smonzon/.opentrons/deck_calibration.json not found. Loading defaults
/home/smonzon/.opentrons/robot_settings.json not found. Loading defaults


In [56]:
tips300 = [
        ctx.load_labware(
            'opentrons_96_tiprack_300ul', slot, '200µl filter tiprack')
        for slot in ['2', '3', '5', '6', '9']
    ]
tips1000 = [
        ctx.load_labware('opentrons_96_filtertiprack_1000ul', slot,
                         '1000µl filter tiprack')
        for slot in ['8', '4']
    ]

m300 = ctx.load_instrument('p300_multi_gen2', 'right', tip_racks=tips300)
p1000 = ctx.load_instrument('p1000_single_gen2', 'left', tip_racks=tips1000)
m300

print(str(m300))
if "P1000" in str(p1000):
    print("yes")

P300 8-Channel GEN2 on right mount
yes


In [55]:
num_cols = math.ceil(NUM_SAMPLES/8)
wash_sets = [reagent_res.wells()[i:i+2] for i in [5, 7, 9]]
print(wash_sets)
dests = magplate.rows()[0][:num_cols]
for wash_set in wash_sets:
    print(wash_set)
    for i, m in enumerate(dests):
        wash_chan = wash_set[i//6]
        print(wash_chan)

[[A6 of reagent reservoir on 7, A7 of reagent reservoir on 7], [A8 of reagent reservoir on 7, A9 of reagent reservoir on 7], [A10 of reagent reservoir on 7, A11 of reagent reservoir on 7]]
[A6 of reagent reservoir on 7, A7 of reagent reservoir on 7]
A6 of reagent reservoir on 7
A6 of reagent reservoir on 7
A6 of reagent reservoir on 7
A6 of reagent reservoir on 7
A6 of reagent reservoir on 7
A6 of reagent reservoir on 7
A7 of reagent reservoir on 7
[A8 of reagent reservoir on 7, A9 of reagent reservoir on 7]
A8 of reagent reservoir on 7
A8 of reagent reservoir on 7
A8 of reagent reservoir on 7
A8 of reagent reservoir on 7
A8 of reagent reservoir on 7
A8 of reagent reservoir on 7
A9 of reagent reservoir on 7
[A10 of reagent reservoir on 7, A11 of reagent reservoir on 7]
A10 of reagent reservoir on 7
A10 of reagent reservoir on 7
A10 of reagent reservoir on 7
A10 of reagent reservoir on 7
A10 of reagent reservoir on 7
A10 of reagent reservoir on 7
A11 of reagent reservoir on 7
